In [1]:
from keras.models import load_model, Model, Sequential
from packages.Utility.CNN.FeatureManager import FeatureExtractor, FeatureReader, FeatureWriter
from packages.Utility.Gabor import GaborFilterBanks, GaborParams, RotatedGaborParams
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Layer, Dense
from packages.Utility.DatasetManager import DatasetReader
from packages.Utility.CNN.FeatureManager import FeatureExtractor, FeatureReader, FeatureWriter
from packages.Utility.Metrics import ConfusionMatrix, CNN_Evaluator, SVM_Evaluator
from sklearn import svm
from packages.Model.CFNet import CFNet1_1

Using TensorFlow backend.


# Inisialisasi

In [25]:
dataset_dir = 'dataset/bin/DG321'
weight_dir = 'trained/digit LN[1]/Fold 1/acc 0.9627 - epoch 122.h5'

# Load Dataset

In [26]:
reader = DatasetReader(dataset_dir)
fold = 0

X_train, Y_train_onehot, Y_train_single = reader.getTrainData(fold)
X_test, Y_test_onehot, Y_test_single = reader.getTestData(fold)

# Mendefinisikan Model

In [2]:
model = CFNet1_1()
model.model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
c1 (Conv2D)                  (None, 32, 32, 32)        896       
_________________________________________________________________
c2 (Conv2D)                  (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 32)        0         
_________________________________________________________________
c3 (Conv2D)                  (None, 16, 16, 64)        18496     
_________________________________________________________________
c4 (Conv2D)                  (None, 16, 16, 64)        36928     
_________________________________________________________________
mp4 (MaxPooling2D)           (None, 8, 8, 64)          0         
_________________________________________________________________
c5 (

# Load Weights

In [31]:
model.load_weights(weight_dir)

# Ekstraksi Fitur

In [32]:
extractor = FeatureExtractor(model, output_layer = 'flatten')
feature_train = extractor.extract(X_train)
feature_test = extractor.extract(X_test)

# Pelatihan SVM

In [ ]:
cls = svm.SVC(kernel = 'linear', max_iter = 50000, C = 100, decision_function_shape='ovo', verbose = True)
cls.fit(feature_train, Y_train_single)

# Evaluasi Model
### Evaluasi untuk CNN

In [34]:
ev_cnn = CNN_Evaluator(model = model, labels = reader.labels)
y_true_train_cnn, y_pred_train_cnn, score_train_cnn = ev_cnn.evaluate(X_train, Y_train_single)
y_true_test_cnn, y_pred_test_cnn, score_train_cnn = ev_cnn.evaluate(X_test, Y_test_single)

### Evaluasi untuk SVM

In [ ]:
ev_svm = SVM_Evaluator(model = cls, labels = reader.labels)
y_true_train_svm, y_pred_train_svm, score_train_svm = ev_svm.evaluate(feature_train, Y_train_single)
y_true_test_svm, y_pred_test_svm, score_test_svm = ev_svm.evaluate(feature_test, Y_test_single)

# Menampilkan Confusion Matrix

In [ ]:
# Confusion Matrix
cfm = ConfusionMatrix((25, 25), 2, 2, reader.labels)
cfm.xylabel_size = 15
cfm.add(y_true_train_cnn, y_pred_train_cnn, 'Data Training CNN')
cfm.add(y_true_test_cnn, y_pred_test_cnn, 'Data Testing CNN')
cfm.add(y_true_train_svm, y_pred_train_svm, 'Data Training SVM')
cfm.add(y_true_test_svm, y_pred_test_svm, 'Data Testing SVM')
cfm.show()